In [1]:
from datasets import load_dataset
import pandas as pd 
import os
import pandas as pd
from faker import Faker
import random

/Users/azeezatmustapha/Desktop/shopper/backend/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You can pick a specific category like 

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Grocery_and_Gourmet_Food", split="full", trust_remote_code=True)
print(dataset[4])


{'main_category': 'Grocery', 'title': 'Messmer Peppermint 25 bags (6er pack)', 'average_rating': 3.5, 'rating_number': 5, 'features': [], 'description': [], 'price': '29.99', 'images': {'hi_res': [None, None], 'large': ['https://m.media-amazon.com/images/I/51tW33sFp6L.jpg', 'https://m.media-amazon.com/images/I/411mYdC5XOL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/51tW33sFp6L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/411mYdC5XOL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN', 'PT01']}, 'videos': {'title': [], 'url': [], 'user_id': []}, 'store': 'Messmer', 'categories': [], 'details': '{"Brand": "Messmer", "Item Form": "Teabags", "Unit Count": "1.00 Count", "Package Information": "Bag", "Number of Pieces": "6", "Is Discontinued By Manufacturer": "No", "Package Dimensions": "13.19 x 10.39 x 3.27 inches; 1.37 Pounds"}', 'parent_asin': 'B06X9DC27H', 'bought_together': None, 'subtitle': None, 'author': None}


In [43]:
filtered_data = []
target_count = 1000

for item in dataset:
    # Check if all required fields exist and are not empty
    has_images = item.get('images') and len(item['images']) > 0
    has_desc = item.get('description') and len(item['description']) > 0
    has_price = item.get('price') and item['price'] != 'None'
    
    if has_images and has_price and has_desc:
        filtered_data.append({
            "name": item['title'],
            "price": item['price'],
            "description": item['description'],
            # Amazon stores images as a list of dicts; we take the 'large' or first URL
            "image_url": item['images'],
        })
        
    if len(filtered_data) >= target_count:
        break

# Create your clean DataFrame
amazon_1000_df = pd.DataFrame(filtered_data)
print(f"Successfully collected {len(amazon_1000_df)} products.")

Successfully collected 1000 products.


In [44]:
len(amazon_1000_df)

1000

In [15]:
#Get the 1000 most ordered product ids from the orders from instacart
ordered_products = pd.read_csv('data/instacart/order_products__prior.csv')
top_ids = ordered_products['product_id'].value_counts().head(1000).index
top_ids

Index([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845,
       ...
        4945, 13966,  7746, 35199, 45190, 18023, 20378, 43014,  3339, 27020],
      dtype='int64', name='product_id', length=1000)

In [40]:
all_products = pd.read_csv('data/instacart/products.csv')

# select products that are most ordered
instacart_products = all_products[all_products['product_id'].isin(top_ids)]

In [50]:
instacart_products = instacart_products.reset_index(drop=True)
amazon_1000_df = amazon_1000_df.reset_index(drop=True)

# concat will align them perfectly row-by-row
mock_products = pd.concat([instacart_products, amazon_1000_df], axis=1)

# Clean up
if "aisle_id" in mock_products.columns:
    mock_products.drop(columns=["aisle_id"], inplace=True)

mock_products.rename(columns={"department_id": "category_id"}, inplace=True)

print(f"Final row count: {len(mock_products)}") 

Final row count: 1000


In [51]:
mock_products.head()

,product_id,product_name,category_id,name,price,description,image_url
0,34,Peanut Butter Cereal,14,PICARAS Galletas Peruanas Bañadas en Chocolate...,15.99,[2 Bags of PICARAS Galletas Bañadas en Chocola...,{'hi_res': ['https://m.media-amazon.com/images...
1,45,European Cucumber,4,Chincoteague Seafood 90944 Vegetable Red Crab ...,73.57,[Vegetable Red Crab Soup is a colorful combina...,"{'hi_res': [None], 'large': ['https://m.media-..."
2,196,Soda,7,"Vintners Best Fruit Wine Base-Rhubarb,128 oz",47.86,[Rhubarb Fruit Wine Base is perfect for making...,{'hi_res': ['https://m.media-amazon.com/images...
3,248,Dried Sweetened Cranberries,19,"Instant Compostable Espresso Capsules, Lungo M...",8.49,[Instant® Compostable Espresso Capsules delive...,{'hi_res': ['https://m.media-amazon.com/images...
4,260,Cantaloupe,4,"TAJ Indian Poha Powa Flattened Rice (Thick, 4-...",17.99,[TAJ Gourmet Foods provides the best of quality.],{'hi_res': ['https://m.media-amazon.com/images...


In [56]:
## Save the mock data to be populated in the database
compression_opts = dict(method='zip',
                        archive_name='mock.csv')
mock_products.to_csv('../main-data/mock.zip', index=False,
          compression=compression_opts)   